In [1]:
# Web scraper for Edmunds
# Selenium is used to handle the javascript on a website

#Importing libraries
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import pandas as pd
import os     #to manipulate files

import warnings
warnings.filterwarnings('ignore')

In [46]:
chrome_path = r"E:\Chrome Driver\chromedriver.exe"
driver = webdriver.Chrome(chrome_path) # This will open up a chrome window

In [47]:
Page = 2342
url = "https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p" + str(Page)

In [48]:
driver.get(url)

In [49]:
userlist = []
datelist = []
postlist = []
quotelist = []
while len(postlist)<5000:
    #User
    users = driver.find_elements_by_class_name('Username')
    for user in users:
        userlist.append(user.text)
    
    #Date
    dates = driver.find_elements_by_class_name('DateCreated')
    for date in dates:
        datelist.append(date.text)
    
    # This is the list of all the comments
    posts = driver.find_elements_by_class_name('Item-Body')
    for post in posts:
        postlist.append(post.text)

    # This is the list of the blockquotes that you don't want
    quotes = driver.find_elements_by_class_name('Quote')
    for quote in quotes:
        quotelist.append(quote.text)
        
    Page = Page - 1                    
    url = "https://forums.edmunds.com/discussion/18576/general/x/edmunds-members-cars-conversations/p" + str(Page)

    driver.get(url)

In [54]:
# Now you have to delete quotes from the posts
for j in range(0,len(quotelist)):
    for i in range(0,len(postlist)):
        if (quotelist[j] in postlist[i]):
            postlist[i]=postlist[i].replace(quotelist[j],'')

In [55]:
df_raw = pd.DataFrame(datelist)
df_raw['Users'] = userlist
df_raw['Text'] = postlist
df_raw.columns = ['Date','User','Text']

In [56]:
postfile_name = r'E:\Extra Projects\Web Scrapping\Posts.csv'
df_raw.to_csv(postfile_name)